# **Motion Detection with Frame Differencing**

In this notebook we will explore a simple method of moving object detection via Frame Differencing.


In [131]:
import os
import sys
from tqdm import tqdm
import numpy as np
import cv2
import random
sys.path.append(os.path.join(os.getcwd(), 'common_utils'))
from common_utils.choose_dataset_and_video import get_random_choice
from common_utils.get_moving_object_detections import get_contour_detections, draw_contours
from common_utils.make_video import merge_images, add_text, make_video

%matplotlib inline

notebook_id = 'TA'

### Read the data

We use the CD2014 or SBMnet dataset.

The following are helper function to choose a random video and load the images in sequence

In [132]:
def get_directories(path):
    """Return a list of directories name on the specifed path"""
    return [file for file in os.listdir(path) if os.path.isdir(os.path.join(path, file))]

In [133]:
def process_folder(dataset_path):
    """Call your executable for all sequences in all categories."""
    video_paths = []
    for category in get_directories(dataset_path):
        category_path = os.path.join(dataset_path, category)
        for video in get_directories(category_path):
            video_path = os.path.join(category_path, os.path.join(video, 'input'))
            video_paths.append(video_path)
    return video_paths

In [134]:
def get_images_from_folder(folder_path):
    # Get a list of all files in the folder
    files = os.listdir(folder_path)
    # Filter for only jpg files and construct the full path for each
    jpg_image_paths = [os.path.join(folder_path, file) for file in files if file.lower().endswith('.jpg')]
    
    return jpg_image_paths

In [135]:
dataset_path, chosen_dataset, random_category, random_sub_category = get_random_choice(notebook_id)
print('Chosen dataset:', chosen_dataset)
print('Chosen category:', random_category)
print('Chosen sub_category:', random_sub_category)

video_folder_path = os.path.join(dataset_path, random_category, random_sub_category, 'input')

image_paths = get_images_from_folder(video_folder_path)
results_folder_path = os.path.join(os.getcwd(), f'datasets\\results\\{chosen_dataset}_results') 

Chosen dataset: CDNet_2014
Chosen category: baseline
Chosen sub_category: highway


## **Get Motion Mask**
In this step we get a thresholded image mask. This image mask will give us relative locations of all moving targets.

In [136]:
def get_mask(frame1, frame2, adaptive=True, block_size=21, constant=5):
    # convert to grayscale
    frame1 = cv2.cvtColor(frame1, cv2.COLOR_RGB2GRAY)
    frame2 = cv2.cvtColor(frame2, cv2.COLOR_RGB2GRAY)

    # Compute the absolute difference between the two frames. This highlights the changes (i.e., movement) between the frames
    frame_diff = cv2.absdiff(frame2, frame1)
    # Apply a median blur with a 3x3 kernel to the frame difference. Blurring helps reduce noise and smooth the image, potentially eliminating isolated fluctuations
    frame_diff = cv2.medianBlur(frame_diff, 3)
    # Apply a simple threshold to create a binary mask of moving pixels
    if adaptive:
        mask = cv2.adaptiveThreshold(frame_diff, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, constant)
    else:
        _, mask = cv2.threshold(frame_diff, 30, 255, cv2.THRESH_BINARY)
    
    return mask

## **Reference Frame Differencing**
### Method
1) Chose a reference frame (the first one in this case)
2) Take Frame Difference: $d = reference frame - frame_{t}$
3) Threshold frame difference to get mask
4) Perform detection on mask via contour finding
5) Draw bounding boxes

In [137]:
# REFERENCE FRAME DIFFERENCING
video_frames_rfd = []
reference_frame = cv2.imread(image_paths[0])
for idx in tqdm(range(1, len(image_paths)), total=len(image_paths), desc="Processing Frames"):
    # read frame
    frame = cv2.imread(image_paths[idx])
    
    # get detections
    mask = get_mask(reference_frame, frame, block_size=29, constant=9)
    detections = get_contour_detections(mask, thresh=400, distance_threshold=13)
    rgb_detections = draw_contours(frame, detections)

    merged_frame = add_text(merge_images(cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB), rgb_detections), 'REFERENCE FRAME DIFFERENCING')
    # Append to list for video
    video_frames_rfd.append(merged_frame)

Processing Frames: 100%|█████████▉| 1699/1700 [00:06<00:00, 271.61it/s]


In [138]:
os.makedirs(os.path.join(results_folder_path, 'Reference Frame Differencing'), exist_ok=True)
make_video(video_frames_rfd, os.path.join(results_folder_path, 'Reference Frame Differencing', f'{random_sub_category}.mp4'))

Processing Frames: 100%|██████████| 1699/1699 [00:02<00:00, 748.92it/s]


## **Adjacent Frame Differencing**
### Method
1) Take Frame Difference: $d = frame_{t+1} - frame_{t}$
2) Threshold frame difference to get mask
3) Perform detection on mask via contour finding
4) Draw bounding boxes

In [139]:
# ADJACENT FRAME DIFFERENCING
video_frames_afd = []
for idx in tqdm(range(1, len(image_paths)), total=len(image_paths), desc="Processing Frames"):
    # read frame
    frame1 = cv2.imread(image_paths[idx - 1])
    frame2 = cv2.imread(image_paths[idx])
    
    # get detections
    mask = get_mask(frame1, frame2, block_size=21, constant=5)
    detections = get_contour_detections(mask, thresh=400, distance_threshold=40)
    rgb_detections = draw_contours(frame2, detections)

    merged_frame = add_text(merge_images(cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB), rgb_detections), 'ADJACENT FRAME DIFFERENCING')

    # Append to list for video
    video_frames_afd.append(merged_frame)

Processing Frames: 100%|█████████▉| 1699/1700 [00:07<00:00, 214.05it/s]


In [140]:
os.makedirs(os.path.join(results_folder_path, 'Adjacent Frame Differencing'), exist_ok=True)
make_video(video_frames_afd, os.path.join(results_folder_path, 'Adjacent Frame Differencing', f'{random_sub_category}.mp4'))

Processing Frames: 100%|██████████| 1699/1699 [00:02<00:00, 831.52it/s]


## **Median Frame Differencing**
### Method
1) Calculate the median of buffered frames (previous n frame) to generate a reference frame
2) Take Frame Difference: $d = median frame - frame_{t}$
3) Threshold frame difference to get mask
4) Perform detection on mask via contour finding
5) Draw bounding boxes

In [141]:
from collections import deque

video_frames_mfd = []
buffer_size = 5
frame_buffer = deque(maxlen=buffer_size)
for idx in tqdm(range(1, len(image_paths)), total=len(image_paths), desc="Processing Frames"):
    frame = cv2.imread(image_paths[idx])
    frame_buffer.append(frame)
    median_frame = np.median(np.array(frame_buffer), axis=0).astype(np.uint8)
    
     # get detections
    mask = get_mask(median_frame, frame)
    detections = get_contour_detections(mask, thresh=500, distance_threshold=55)
    rgb_detections = draw_contours(frame, detections)

    merged_frame = add_text(merge_images(cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB), rgb_detections), 'MEDIAN FRAME DIFFERENCING')

    # Append to list for video
    video_frames_mfd.append(merged_frame)

Processing Frames: 100%|█████████▉| 1699/1700 [00:24<00:00, 68.72it/s]


In [142]:
os.makedirs(os.path.join(results_folder_path, 'Median Frame Differencing'), exist_ok=True)
make_video(video_frames_mfd, os.path.join(results_folder_path, 'Median Frame Differencing', f'{random_sub_category}.mp4'))

Processing Frames: 100%|██████████| 1699/1699 [00:02<00:00, 720.61it/s]


## **Median Frame Differencing with Morphology**
### Method
1) Calculate the median of buffered frames (previous n frame) to generate a reference frame
2) Take Frame Difference: $d = median frame - frame_{t}$
3) Threshold frame difference to get mask
4) Perform Morphological operations to reduce noise and make the moving objects stand out more
5) Perform detection on mask via contour finding
6) Draw bounding boxes

In [143]:
from collections import deque

video_frames_mfd_morph = []
buffer_size = 5
frame_buffer = deque(maxlen=buffer_size)
# Kernel for morphological operations
kernel = np.ones((3, 3), np.uint8)

for idx in tqdm(range(1, len(image_paths)), total=len(image_paths), desc="Processing Frames"):
    frame = cv2.imread(image_paths[idx])
    frame_buffer.append(frame)
    median_frame = np.median(np.array(frame_buffer), axis=0).astype(np.uint8)
    
    mask = get_mask(median_frame, frame, True)
    # Closing to fill small holes
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    detections = get_contour_detections(mask, thresh=400, distance_threshold=10)
    rgb_detections = draw_contours(frame, detections)

    merged_frame = add_text(merge_images(cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB), rgb_detections), 'MEDIAN FRAME DIFFERENCING WITH MORPHOLOGY')

    # Append to list for video
    video_frames_mfd_morph.append(merged_frame)

Processing Frames: 100%|█████████▉| 1699/1700 [00:24<00:00, 69.39it/s]


In [144]:
os.makedirs(os.path.join(results_folder_path, 'Median Frame Differencing with Morph'), exist_ok=True)
make_video(video_frames_mfd_morph, os.path.join(results_folder_path, 'Median Frame Differencing with Morph', f'{random_sub_category}.mp4'))

Processing Frames: 100%|██████████| 1699/1699 [00:03<00:00, 561.11it/s]


## **Mixture of Gaussian**

In [145]:
video_frames_mog = []
kernel = np.ones((2, 2), np.uint8)
background_model = cv2.createBackgroundSubtractorMOG2(history=200, varThreshold=50)

for idx in tqdm(range(1, len(image_paths)), total=len(image_paths), desc="Processing Frames"):
    frame = cv2.imread(image_paths[idx])
    
    mask = background_model.apply(frame)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    detections = get_contour_detections(mask, thresh=200, distance_threshold=25)
    rgb_detections = draw_contours(frame, detections)

    merged_frame = add_text(merge_images(cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB), rgb_detections), 'MIXTURE OF GAUSSIAN')

    # Append to list for video
    video_frames_mog.append(merged_frame)

Processing Frames: 100%|█████████▉| 1699/1700 [00:10<00:00, 164.56it/s]


In [146]:
os.makedirs(os.path.join(results_folder_path, 'Mixture of Gaussian'), exist_ok=True)
make_video(video_frames_mog, os.path.join(results_folder_path, 'Mixture of Gaussian', f'{random_sub_category}.mp4'))

Processing Frames: 100%|██████████| 1699/1699 [00:02<00:00, 652.95it/s]
